# Lab 3: Word Embeddings and Language Modelling

Adam Ek

In this lab we'll explore constructing *static* word embeddings (i.e. word2vec) and building language models. We'll also evaluate these systems on intermediate tasks, namely word similarity and identifying "good" and "bad" sentences.

* For this we'll use pytorch. Some basic operations that will be useful can be found here: https://jhui.github.io/2018/02/09/PyTorch-Basic-operations
* In general: we are not interested in getting state-of-the-art performance :) focus on the implementation and not results of your model. For this reason, you can use a subset of the dataset: the first 5000-10 000 sentences or so, on linux/mac: ```head -n 10000 inputfile > outputfile```. 
* If possible, use the MLTGpu, it will make everything faster :)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# for gpu, replace "cpu" with "cuda:n" where n is the index of the GPU
device = torch.device('cpu')

# Word2Vec embeddings

In this first part we'll construct a word2vec model which will give us *static* word embeddings (that is, they are fixed after training).

After we've trained our model we will evaluate the embeddings obtained on a word similarity task.

## Formatting data


First we need to load some data, you can download the file on canvas under files/03-lab-data/wiki-corpus.txt. The file contains 50 000 sentences randomly selected from the complete wikipedia. Each line in the file contains one sentence. The sentences are whitespace tokenized.

Your first task is to create a dataset suitable for word2vec. That is, we define some ```window_size``` then iterate over all sentences in the dataset, putting the center word in one field and the context words in another (separate the fields with ```tab```).

For example, the sentece "this is a lab" with ```window size = 4``` will be formatted as:
```
center, context
---------------------
this    is a lab
is      this a lab
a       this is lab
lab     this is a
```

this will be our training examples when training the word2vec model.

[3 marks]

In [2]:
data_path = 'data/wiki-corpus.txt'
WINDOW_SIZE = 4 #on each side!
def corpus_reader(data_path, output="data/my_data.csv", k=WINDOW_SIZE):
    
    with open(data_path, mode="r") as f:
        my_data=[line.split(" ") for line in f.read().split("\n")]
    
    #print(my_data[:10])
    my_data=my_data[:1000] #OBS!
    
    my_string=""
    for sentence in my_data:
        if len(sentence)>1: #only at least two-word sentences (minmal context)
            my_context=["<x>"]*k + sentence + ["<x>"]*k
            i=k
            for w in sentence:
                my_string+=w+"\t"
                j=i+1
                left=my_context[i-k:i]
                right=my_context[j:j+k]
                context_words=[c for c in left+right if c not in["<x>", "(", ")", ".", "!", ",", '"', "'", "?"]]
                my_string+=" ".join(context_words)+"\n"
                i+=1
    
    #so what do I do with my_string????
    with open(output, mode="w") as f:
        f.write(my_string[:-1]) #note to self: we do not want empty rows

corpus_reader(data_path)


We sampled 50 000 senteces completely random from the *whole* wikipedia for our training data. Give some reasons why this is good, and why it might be bad. (*note*: We'll have a few questions like these, one or two reasons for and against is sufficient)

[2 marks]

**ANSWER**


### Loading the data

We now need to load the data in an appropriate format for torchtext (https://torchtext.readthedocs.io/en/latest/). We'll use PyText for this and it'll follow the same structure as I showed you in the lecture (remember to lower-case all tokens). Create a function which returns a (bucket)iterator of the training data, and the vocabulary object (```Field```). 

(*hint1*: you can format the data such that the center word always is first, then you only need to use one field)

(*hint2*: the code I showed you during the leture is available in /files/pytorch_tutorial/ on canvas)

[4 marks]

In [ ]:
from torchtext.legacy.data import Field, BucketIterator, Iterator, TabularDataset

def get_data(
    my_path = "data/my_data.csv",
    batch_size = 3
    ):
    
    whitespacer = lambda x: x.split(' ') #from: files/partofspeech-tagging_main.py on Canvas
   
    MY_FIELD = Field(
        tokenize = whitespacer,
        lower=True,
        batch_first = True
        )
        
    my_fields = [("target_word", MY_FIELD), ("context_words", MY_FIELD)]   
    
    train = TabularDataset(
        path   = my_path,
        format = 'csv',
        fields = my_fields,
        #skip_header = True,
        csv_reader_params = {'delimiter':'\t', 'quotechar':'{'} #Note "{" is used as quotechar
        )
    
    #Note on quote character selection: 
    #By trial and error I found that "{" is not a character in the corpus.
    #Therefore it can be used as quote character. (Swedish "å", "ä" and "ö" are in 
    #the corpus. So is "½".)


    MY_FIELD.build_vocab(train)
    
    my_bucket = BucketIterator(
        train,      
        batch_size        = batch_size,
        sort_within_batch = True,
        sort_key          = lambda x: len(x.context_words),
        shuffle           = True,
        device            = device)    
    
    
    
    return my_bucket, MY_FIELD.vocab


In [ ]:
#FOR DEVELOPMENT; PLEASE IGNORE! NO HARM IN RUNNING IT THOUGH.
b, v = get_data()
print(b)
print(v)
print(len(v))

In [ ]:
#FOR DEVELOPMENT; PLEASE IGNORE! (DOES NOT AFFECT CODE BELOW)
t=0
c=0

for i, x in enumerate(b):
    if x.target_word.shape[0] != 3:
        t+=1
        print(x.target_word)
    if x.context_words.shape[0] != 3:
        c+=1
        print(x.context_words)

print("t ", t)
print("c ", c)

We lower-cased all tokens above; give some reasons why this is a good idea, and why it may be harmful to our embeddings.

[2 marks]

**ANSWER:** 
Here is a question to answer this question: Does upper case carry any relevant information? If so, which?

Different answers to these question can guide our discussion of reasons for and against lower-casing. 

**Yes**, when lower-casing, at least two types of information that are *lost*:
*    Named entities (e.g. *Shell* vs *shell*)
*    Sentence bounary information (e.g. *The* is a better predicition after *.* than *the*).

In as far as such information is important for the task we are trying to solve with the embeddings, lower-casing can be "harmfull". Consider, for example, named entitiy recognition and language modelling aiming to predict sequentiality of language.

Alternatively, there is also a sense in which: **no**, upper case does not carry much relevant information; mostly, the upper vs. lower case is *semantically* redundant, but *expressively* expensive. Potentially our vocabulary could be twice as long. As such, there is good reason for lower-casing. 

## Word Embeddings Model

We will implement the CBOW model for constructing word embedding models.

In [ ]:
import torch.optim as optim

In the CBOW model we try to predict the center word based on the context. That is, we take as input ```n``` context words, encode them as vectors, then combine them by summation. This will give us one embedding. We then use this embedding to predict *which* word in our vocabuary is the most likely center word. 

Implement this model 

[7 marks]

In [ ]:
class CBOWModel(nn.Module):
    def __init__(self, voc_size, hidden_d):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(voc_size, hidden_d) #voc_size * window_size?
        self.prediction = nn.Linear(hidden_d, voc_size)
        self.vsize=voc_size #I have used this below
        
        # NOTE: Softmax is part of the loss function implemented below
    
    def forward(self, context):
        #print(context)
        embedded_context = self.projection_function(context)
        #print(embedded_context)
        projection = self.embeddings(embedded_context) 
        predictions = self.prediction(projection)
        
        return predictions
    
        
    def projection_function(self, xs):
        """
        This function will take as input a tensor of size (B, S, D)
        where B is the batch_size, S the window size, and D the dimensionality of embeddings
        this function should compute the sum over the embedding dimensions of the input, 
        that is, we transform (B, S, D) to (B, 1, D) or (B, D) 
        """
        #Collapsed one-hot vector representation... 
        b=xs.shape[0] #The number of "lines" (the batch size)
        
        xs_sum=torch.zeros(b, self.vsize, dtype=torch.int) 
        for i in range(b):
            ctxt=xs[i] #context of ith batch
            for j in ctxt:
                xs_sum[i][j]+=1
     
        #xs_sum = torch.sum(xs, dim=0) 
        #column-wise sum for the n=S embeddings  x  n=B batches
        #https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be
        return xs_sum

Now we need to train the models. First we define which hyperparameters to use. (You can change these, for example when *developing* your model you can use a batch size of 2 and a very low dimensionality (say 10), just to speed things up). When actually training your model *fo real*, you can use a batch size of [8,16,32,64], and embedding dimensionality of [128,256].

In [ ]:
# you can change these numbers to suit your needs :)
word_embeddings_hyperparameters = {'epochs':3,
                                   'batch_size':8,
                                   'embedding_size':128,
                                   'learning_rate':0.001,
                                   'embedding_dim':128}

Train your model. Iterate over the dataset, get outputs from your model, calculate loss and backpropagate.

We mentioned in the lecture that we use Negative Log Likelihood (https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html) loss to train Word2Vec model. In this lab we'll take a shortcut when *training* and use Cross Entropy Loss (https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), basically it combines ```log_softmax``` and ```NLLLoss```. So what your model should output is a *score* for each word in our vocabulary. The ```CrossEntropyLoss``` will then assign probabilities and calculate the negative log likelihood loss.

[3 marks]

In [ ]:

# load data

dataset, vocab = get_data(
    #batch_size = word_embeddings_hyperparameters['batch_size']
    batch_size = 3
    )
print("Length of vocabulary: ", len(vocab))

# build model and construct loss/optimizer
cbow_model = CBOWModel(len(vocab), word_embeddings_hyperparameters['embedding_dim'])
cbow_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=word_embeddings_hyperparameters['learning_rate'])

# start training loop
cbow_model.train()
total_loss = 0
for epoch in range(word_embeddings_hyperparameters['epochs']):
    for i, batch in enumerate(dataset):
        
        context = batch.context_words
        target_word = batch.target_word
        
        #print(context)
        #print(target_word)
        
        # send your batch of sentences to the model
        output = cbow_model(context)
        
        # compute the loss, you'll need to reshape the input
        # you can read more about this is the documentation for
        # CrossEntropyLoss
        
        #print(output)
        hot_output=torch.zeros(target_word.shape[0], len(vocab), dtype=torch.long)
        for i in range(target_word.shape[0]):
            indx=target_word[i]
            hot_output[i][indx]+=1

        loss = loss_fn(output, hot_output)
        total_loss += loss.item()
        
        # print average loss for the epoch
        print(total_loss/(i+1), end='\r') 
        
        # compute gradients
        loss.backward()
        
        # update parameters
        optimizer.step()
        
        # reset gradients
        optimizer.zero_grad

    print()
        

In [ ]:
#SAVING THE MODEL
PATH = "models/my_model.pt"
torch.save(cbow_model, PATH)

In [ ]:
#LOAD MODEL
model = torch.load(PATH)
model.eval()

In [ ]:
#FOR DEVELOPMENT; PLEASE IGNORE
e=cbow_model.embeddings.weight
print(len(e))
#print(e[23])

wookie=vocab.itos
wookie[0]

## Evaluating the model

We will evaluate the model on a dataset of word similarities, WordSim353 (http://alfonseca.org/eng/research/wordsim353.html , also avalable in vanvas under files/03-l). The first thing we need to do is read the dataset and translate it to integers. What we'll do is to reuse the ```Field``` that records word indexes (the second output of ```get_data()```) and use it to parse the file.

The wordsim data is structured as follows:

```
word1 word2 score
...
```


The ```Field``` we got from ```read_data()``` has two built-in functions, ```stoi``` which maps a string to an integer and ```itos``` which maps an integer to a string. 

What our datareader needs to do is: 

```
for line in file:
    word1, word2, score = file.split()
    # encode word1 and word2 as integers
    word1_idx = vocab.vocab.stoi[word1]
    word2_idx = vocab.vocab.stoi[word2]
```

when we have the integers for ```word_1``` and ```word2``` we'll compute the similarity between their word embeddings with *cosine simlarity*. We can obtain the embeddings by querying the embedding layer of the model.

We calculate the cosine similarity for each word pair in the dataset, then compute the pearson correlation between the similarities we obtained with the scores given in the dataset. 

[4 marks]

In [ ]:
# your code goes here

def read_wordsim(path, 
                 vocabulary=vocab.stoi, 
                 checker=vocab.itos,
                 embeddings=cbow_model.embeddings.weight):
    #https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
    dataset_sims = []
    model_sims = []
    with open(path) as f:
        for line in f.read().split("\n"):
            if len(line) != 0: #We do not want empty lines.
                word1, word2, score = line.split() #splits by space by default

            # get the index for the word
            word1_idx = vocabulary[word1]
            word2_idx = vocabulary[word2]            
            
            if (checker[word1_idx]=='<unk>', checker[word2_idx]=='<unk>') == (False, False):
            #If two words are unknown to the model they will both be assigned "<unk>"
            #and associated with the same embedding (u). The cosine similarity of u and u
            #will be 1. I suggest that such scores should not be part of our evaluation.

                score = float(score)
                dataset_sims.append(score)

                #print(word1, " -- ", word2)
                #print(word1_idx)
                #print(word2_idx)

                # get the embedding of the word
                # the hidden layer will be a matrix of weights;
                # use the index to identify the right row/column (?)
                word1_emb = embeddings[word1_idx]
                word2_emb = embeddings[word2_idx]

                # compute cosine similarity, we'll use the version included in pytorch functional
                # https://pytorch.org/docs/master/generated/torch.nn.functional.cosine_similarity.html
                cosine_similarity = F.cosine_similarity(word1_emb, word2_emb, dim=0)
                
                #In order to identify "best" and "worst performing word pairs"; see below.
                print(f"{word1} -- {word2} (psy: {round(score/10, 3)}, model: {round(cosine_similarity.item(), 3)}).")

                model_sims.append(cosine_similarity.item())
    
    return dataset_sims, model_sims

path = 'eval_data/wordsim_similarity_goldstandard.txt'
data, model = read_wordsim(path)
pearson_correlation = np.corrcoef(data, model)
            
# the non-diagonals give the pearson correlation,
print("\nRESULTS:")
print(pearson_correlation)

Do you think the model performs good or bad? Why?

[3 marks]

**ANSWER**


Select the 10 best and 10 worst performing word pairs, can you see any patterns that explain why *these* are the best and worst word pairs?

[3 marks]

**ANSWER**


Suggest some ways of improving the model we apply to WordSim353.

[3 marks]

**ANSWER**


If we consider a scenario where we use these embeddings in a downstream task, for example sentiment analysis (roughly: determining whether a sentence is positive or negative). 

Give some examples why the sentiment analysis model would benefit from our embeddnings and one examples why our embeddings could hur the performance of the sentiment model.

[3 marks]

**ANSWER**


# Language modeling

In this second part we'll build a simple LSTM language model. Your task is to construct a model which takes a sentence as input and predict the next word for each word in the sentence. For this you'll use the ```LSTM``` class provided by PyTorch (https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html). You can read more about the LSTM here: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

NOTE!!!: Use the same dataset (wiki-cropus.txt) as before.

Our setup is similar to before, we first encode the words as distributed representations then pass these to the LSTM and for each output we predict the next word.

For this we'll build a new dataloader with torchtext, the file we pass to the dataloader should contain one sentence per line, with words separated by whitespace.

```
word_1, ..., word_n
word_1, ..., word_k
...
```

in this dataloader you want to make sure that each sentence begins with a ```<start>``` token and ends with a ```<end>``` token, there is a keyword argument in ```Field``` for this :). But other than that, as before you read the dataset and output a iterator over the dataset and a vocabulary. 

Implement the dataloader, language model and the training loop (the training loop will basically be the same as for word2vec).

[12 marks]

In [ ]:
# you can change these numbers to suit your needs as before :)
lm_hyperparameters = {'epochs':3,
                      'batch_size':16,
                      'learning_rate':0.001,
                      'embedding_dim':128,
                      'output_dim':128}

In [3]:
#FOR DEVELOPMENT ONLY; PLEASE IGNORE.

def shorty(txt_in, txt_out, max_lines=1000):
    with open(txt_in, mode="r") as f:
        my_data = f.read().split("\n")
    with open(txt_out, mode="w") as f:
        output="\n".join(my_data[:max_lines])
        f.write(output)
shorty('data/wiki-corpus.txt', 'data/wiki-corpus-short.txt')

In [ ]:
#data_path = 'data/wiki-corpus.txt'
data_path = 'data/wiki-corpus-short.txt' #OBS!

from torchtext.legacy.data import Field, BucketIterator, Iterator, TabularDataset

def get_data(
    my_path = data_path,
    batch_size = 3
    ):
    
    whitespacer = lambda x: x.split(' ') #from: files/partofspeech-tagging_main.py on Canvas
    
    MY_FIELD = Field(
        tokenize = whitespacer,
        lower=True,
        batch_first = True,
        init_token="<start>", 
        eos_token="<end>"
    )

    train = TabularDataset(
        path   = my_path,
        #train  = file,
        format = 'csv',
        fields = [("sentence", MY_FIELD)],
        #skip_header = True,
        #csv_reader_params = {'delimiter':'\t', 'quotechar':'ö'} #supposedly 'ö' is not in our English data
    )

    MY_FIELD.build_vocab(train)
    
    my_bucket = BucketIterator(
        train,      
        batch_size        = batch_size,
        sort_within_batch = True,
        sort_key          = lambda x: len(x.sentence),
        shuffle           = True,
        device            = device)    
    
    return my_bucket, MY_FIELD.vocab


In [ ]:
#FOR DEVELOPMENT ONLY; PLEASE IGNORE!
b, v = get_data()
len(v)
for i in [0,1,2]: 
    print(v.itos[i])


In [ ]:
class LM_withLSTM(nn.Module):
    def __init__(self, n_words, emb_dim, outp_dim):
        super(LM_withLSTM, self).__init__()
        self.embeddings = nn.Embedding(n_words, emb_dim)
        self.LSTM = nn.LSTM(emb_dim, outp_dim, batch_first=True)
        self.predict_word = nn.Linear(outp_dim, n_words)
    
    def forward(self, seq):
        embedded_seq = self.embeddings(seq)
        timestep_reprentation, *_ = self.LSTM(embedded_seq)
        predicted_words = self.predict_word(timestep_reprentation)
        
        return predicted_words

In [ ]:
import torch.optim as optim
# load data
#dataset, vocab = get_data(batch_size = lm_hyperparameters["batch_size"])
dataset, vocab = get_data()

# build model and construct loss/optimizer
lm_model = LM_withLSTM(len(vocab), 
                       lm_hyperparameters['embedding_dim'],
                       lm_hyperparameters['output_dim'])
lm_model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(lm_model.parameters(), lr=lm_hyperparameters['learning_rate'])
#here cbow model was provided . i changed to lm model ----

# start training loop
total_loss = 0
for epoch in range(lm_hyperparameters['epochs']):
    for i, batch in enumerate(dataset):
        
        # the strucure for each BATCH is:
        # <start>, w0, ..., wn, <end>
        sentence = batch.sentence
        #print(sentence)
        
        # when training the model, at each input we predict the *NEXT* token
        # consequently there is nothing to predict when we give the model 
        # <end> as input. 
        # thus, we do not want to give <end> as input to the model, select 
        # from each batch all tokens except the last. 
        # tip: use pytorch indexing/slicing (same as numpy) 
        # (https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html#operations-on-tensors)
        # (https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/)
        input_sentence = sentence[:, :-1]
        #print(input_sentence)
        
        # send your batch of sentences to the model
        output = lm_model(input_sentence)
        #print(output)
        
        # for each output, the model predict the NEXT token, so we have to reshape 
        # our dataset again. On timestep t, we evaluate on token t+1. That is,
        # we never predict the <start> token ;) so this time, we select all but the first 
        # token from sentences (that is, all the tokens that we predict)
        gold_data = sentence[:, 1:]
        #print(gold_data)
        
        #BUILDING ONE-HOT VECTORS
        hot_gold=torch.zeros(gold_data.shape[0], len(vocab), dtype=torch.long)
        for i in range(gold_data.shape[0]):
            for j in gold_data[i]:
                hot_gold[i][j]+=1
        
        # the shape of the output and sentence variable need to be changed,
        # for the loss function. Details are in the documentation.
        # You can use .view(...,...) to reshape the tensors  
        loss = loss_fn(output, hot_gold)
        #print(loss)
        total_loss += loss.item()
        
        print(total_loss/(i+1), end='\r') 
        
        # print average loss for the epoch
        #print(total_loss/(i+1), end='\r') 
      
        # compute gradients
        loss.backward()
        
        # update parameters
        optimizer.step()
        
        # reset gradients
        optimizer.zero_grad
        
        #break
        
    print()

In [ ]:
#FOR SAVING MODEL
PATH = "models/my_LSTMmodel.pt"
torch.save(lm_model, PATH)

In [ ]:
#FOR LOADING MODEL
my_lm_model = torch.load(PATH)
my_lm_model.eval()

### Evaluating the language model

We'll evaluate our model using the BLiMP dataset (https://github.com/alexwarstadt/blimp). The BLiMP dataset contains sets of linguistic minimal pairs for various syntactic and semantic phenomena, We'll evaluate our model on *existential quantifiers* (link: https://github.com/alexwarstadt/blimp/blob/master/data/existential_there_quantifiers_1.jsonl). This data, as the name suggests, investigate whether language models assign higher probability to *correct* usage of there-quantifiers. 

An example entry in the dataset is: 

```
{"sentence_good": "There was a documentary about music irritating Allison.", "sentence_bad": "There was each documentary about music irritating Allison.", "field": "semantics", "linguistics_term": "quantifiers", "UID": "existential_there_quantifiers_1", "simple_LM_method": true, "one_prefix_method": false, "two_prefix_method": false, "lexically_identical": false, "pairID": "0"}
```

Download the dataset and build a datareader (similar to what you did for word2vec). The dataset structure you should aim for is (you don't need to worry about the other keys for this assignment):

```
good_sentence_1, bad_sentence_1
...
```

your task now is to compare the probability assigned to the good sentence with to the probability assigned to the bad sentence. To compute a probability for a sentence we consider the product of the probabilities assigned to the *gold* tokens, remember, at timestep ```t``` we're predicting which token comes *next* e.g. ```t+1``` (basically, you do the same thing as you did when training).

In rough pseudo code what your code should do is:

```
accuracy = []
for good_sentence, bad_sentence in dataset:
    gs_lm_output = LanguageModel(good_sentence)
    gs_token_probabilities = softmax(gs_lm_output)
    gs_sentence_probability = product(gs_token_probabilities[GOLD_TOKENS])

    bs_lm_output = LanguageModel(bad_sentence)
    bs_token_probabilities = softmax(bs_lm_output)
    bs_sentence_probability = product(bs_token_probabilities[GOLD_TOKENS])

    # int(True) = 1 and int(False) = 0
    is_correct = int(gs_sentence_probability > bs_sentence_probability)
    accuracy.append(is_correct)

print(numpy.mean(accuracy))
    
```

[6 marks]

In [ ]:
# your code goes here

import json

def evaluate_model(path, vocab, model):
    
    accuracy = []
    with open(path) as f:
        # iterate over one pair of sentences at a time
        for line in f:
            # load the data
            data = json.loads(line)
            good_s = data['sentence_good']
            bad_s = data['sentence_bad']
            #print(good_s)
            
            # the data is tokenized as whitespace
            tok_good_s = [token.lower().replace(".", "") for token in good_s.split()]
            tok_bad_s = [token.lower().replace(".", "") for token in bad_s.split()]
            #THERE IS A FULL STOP (.) ON THE LAST TOKEN WHICH WE NEED TO REMOVE
            
            #print(tok_good_s)
            #print(len(tok_good_s))            
            
            # encode your words as integers using the vocab from the dataloader, size is (S)
            # we use unsqueeze to create the batch dimension 
            # in this case our input is only ONE batch, so the size of the tensor becomes: 
            # (S) -> (1, S) as the model expects batches
            enc_good_s = torch.tensor([vocab.stoi[x] for x in tok_good_s], device=device).unsqueeze(0)
            enc_bad_s = torch.tensor([vocab.stoi[x] for x in tok_bad_s], device=device).unsqueeze(0)
            #print(enc_good_s)
            
            # pass your encoded sentences to the model and predict the next tokens
            good_s_pred = model(enc_good_s)
            bad_s_pred = model(enc_bad_s)
            #print(good_s_pred.shape)
            
            # get probabilities with softmax
            gs_probs = F.softmax(good_s_pred[0], dim=1)
            bs_probs = F.softmax(bad_s_pred[0], dim=1)
            #print("probs:\n", gs_probs.shape, "\n", gs_probs)
            
            # select the probability of the gold tokens
            gs_sent_prob = find_token_probs(gs_probs, enc_good_s)
            bs_sent_prob = find_token_probs(bs_probs, enc_bad_s)
            
            accuracy.append(int(gs_sent_prob>bs_sent_prob))
            
    return accuracy
            
def find_token_probs(model_probs, encoded_sentence):
    prob=1
    for counter, idx in enumerate(encoded_sentence[0]):
        prob *= model_probs[counter][idx]
    return prob     
    
path = 'eval_data/existential_there_quantifiers_1.jsonl'
accuracy = evaluate_model(path, vocab, my_lm_model)

print('Final accuracy:')
print(np.round(np.mean(accuracy), 3))


### Analysis

Our model get some score, say, 55% correct predictions. Is this good? Suggest some *baseline* (i.e. a stupid "model" we hope ours is better than) we can compare the model against.

[3 marks]

**ANSWER**

Really Stupid: 1/len(voc)^len(sentence)
Shannon's ... ???

Suggest some improvements you could make to your language model.

[3 marks]

**ANSWER**


Suggest some other metrics we can use to evaluate our system

[2 marks]

**ANSWER**

???perplexity

# Literature


Neural architectures:
* Y. Bengio, R. Ducharme, P. Vincent, and C. Janvin. A neural probabilistic language model. (Links to an external site.) Journal of Machine Learning Research, 3(6):1137–1155, 2003. (Sections 3 and 4 are less relevant today and hence you can glance through them quickly. Instead, look at the Mikolov papers where they describe training word embeddings with the current neural network architectures.)
* T. Mikolov, K. Chen, G. Corrado, and J. Dean. Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781, 2013.
* T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.
    


Total marks: 63